# Grid SST Workflow

Execute the entire Grid SST workflow: input -> pre-process -> gp-regression -> ?

Improvements: 
- Have preprocesing return a value to report on success of operations.

In [1]:
# Standard imports
from pathlib import Path

# Third-party imports
import dask
from dask.distributed import Client, LocalCluster, progress

# Local imports
from ipynb.fs.full.preprocess import run_preprocessing

In [2]:
# Initialize Dask client
cluster = LocalCluster()
client = Client(cluster)
client

/srv/conda/envs/notebook/lib/python3.10/contextlib.py:142: UserWarning: Creating scratch directories is taking a surprisingly long time. This is often due to running workers on a network file system. Consider specifying a local-directory to point workers to write scratch data to a local disk.
  next(self.gen)


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/nikki-t/proxy/8787/status,
Dashboard: /user/nikki-t/proxy/8787/status,Workers: 4
Total threads: 4,Total memory: 15.35 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35507,Workers: 4
Dashboard: /user/nikki-t/proxy/8787/status,Total threads: 4
Started: Just now,Total memory: 15.35 GiB
Comm: tcp://127.0.0.1:41259,Total threads: 1
Dashboard: /user/nikki-t/proxy/35743/status,Memory: 3.84 GiB
Nanny: tcp://127.0.0.1:37611,


In [3]:
# Constants
# OUT = Path("/home/jovyan/shared-readwrite/workflow_data/preprocess")
OUT = Path("/home/jovyan/data")

In [4]:
# Input data required to access and clean data
job_inputs = [
    {
        'short_name': 'VIIRS_NPP-STAR-L2P-v2.80',
        'provider': 'POCLOUD',
        'temporal_range': '2022-07-18T00:00:00Z,2022-07-24T23:59:59Z',
        'bbox': "-40,0,-35,5",
        'qc_threshold': 4,
        'discards': ['satellite_zenith_angle', 'sea_ice_fraction', 'sst_gradient_magnitude', 'sst_front_position']
    },
    {
        'short_name': 'AVHRRF_MB-STAR-L2P-v2.80',
        'provider': 'POCLOUD',
        'temporal_range': '2022-07-18T00:00:00Z,2022-07-24T23:59:59Z',
        'bbox': "-40,0,-35,5",
        'qc_threshold': 3,
        'discards': ['sses_bias', 'sses_standard_deviation', 'sea_ice_fraction', 'sst_front_position', 'l2p_flags', 'sst_gradient_magnitude']
    }
]

In [5]:
# Gather dask "jobs"
lazy_results = []
for job in job_inputs:
    lazy_result = dask.delayed(run_preprocessing)(job['short_name'], job['provider'], job['temporal_range'], job['bbox'], job['qc_threshold'], job['discards'], OUT)
    lazy_results.append(lazy_result)

In [6]:
%%time
# Execute Dask "jobs" (1 thread per for loop iteration)
dask.compute(*lazy_results)

100%|██████████| 31/31 [05:35<00:00, 10.83s/it]


CPU times: user 18.4 s, sys: 3.19 s, total: 21.6 s
Wall time: 5min 48s


(None, None)

In [7]:
print(f"Completed pre-processing. Zarr and NetCDF files can be found in: {OUT}")

Completed pre-processing. Zarr and NetCDF files can be found in: /home/jovyan/data
